In [77]:
from collections import Counter
import pandas as pd

In [65]:
with open('breast_cancer_relations.csv', 'r') as f:
    breast_cancer_relations = f.read().split('\n')
    breast_cancer_relations = [term.strip() for term in breast_cancer_relations]

while True:
    term = input('Enter relation: ')
    if term == 'TERMINATE':
        break
    breast_cancer_relations.append(term.lower())
    
with open('breast_cancer_relations.csv', 'w') as f:
    f.write('\n'.join(breast_cancer_relations))

Enter relation: TERMINATE


In [66]:
len(breast_cancer_relations)

139

In [51]:
with open('cardiovascular_disease_relations.csv', 'r') as f:
    cardiovascular_disease_relations = f.read().split('\n')
    cardiovascular_disease_relations = [term.strip() for term in cardiovascular_disease_relations]

while True:
    term = input('Enter relation: ')
    if term == 'TERMINATE':
        break
    cardiovascular_disease_relations.append(term.lower())
    
with open('cardiovascular_disease_relations.csv', 'w') as f:
    f.write('\n'.join(cardiovascular_disease_relations))

Enter relation: TERMINATE


In [52]:
len(cardiovascular_disease_relations)

424

In [53]:
with open('neurological_disease_relations.csv', 'r') as f:
    neurological_disease_relations = f.read().split('\n')
    neurological_disease_relations = [term.strip() for term in neurological_disease_relations]

while True:
    term = input('Enter relation: ')
    if term == 'TERMINATE':
        break
    neurological_disease_relations.append(term.lower())
    
with open('neurological_disease_relations.csv', 'w') as f:
    f.write('\n'.join(neurological_disease_relations))

Enter relation: TERMINATE


In [54]:
len(neurological_disease_relations)

334

In [55]:
with open('blood_cancer_relations.csv', 'r') as f:
    blood_cancer_relations = f.read().split(',')
    blood_cancer_relations = [term.strip() for term in blood_cancer_relations]

while True:
    term = input('Enter relation: ')
    if term == 'TERMINATE':
        break
    blood_cancer_relations.append(term.lower())
    
with open('blood_cancer_relations.csv', 'w') as f:
    f.write('\n'.join(blood_cancer_relations))

Enter relation: TERMINATE


In [41]:
len(temp)

303

In [56]:
with open('lung_cancer_relations.csv', 'r') as f:
    lung_cancer_relations = f.read().split('\n')
    lung_cancer_relations = [term.strip() for term in lung_cancer_relations]

while True:
    term = input('Enter relation: ')
    if term == 'TERMINATE':
        break
    lung_cancer_relations.append(term.lower())
    
with open('lung_cancer_relations.csv', 'w') as f:
    f.write('\n'.join(lung_cancer_relations))

Enter relation: TERMINATE


In [57]:
len(lung_cancer_relations)

374

In [60]:
diseases = [
    'breast_cancer',
    'cardiovascular_disease',
    'neurological_disease',
    'blood_cancer',
    'lung_cancer'
]
relations = []
for disease in diseases:
    with open(f'{disease}_relations.csv', 'r') as f:
        disease_relation = f.read().split('\n')
        relations += disease_relation
with open(f'combined_disease_relations.csv', 'w') as f:
    f.write('\n'.join(relations))

In [67]:
combined_relations = []
with open(f'combined_disease_relations.csv', 'r') as f:
    combined_relations = f.read().split('\n')

In [79]:
relation_counts = dict(Counter(combined_relations))
relation_counts_df = pd.DataFrame({
    'relation': list(relation_counts.keys()),
    'count': list(relation_counts.values())
})
relation_counts_df.to_excel('relation_counts.xlsx')

In [123]:
# comparison for manually extracted relations vs CRO terms

# unweighted i.e. unique terms
relations_translations = pd.read_excel('relation_counts_rough_translations.xlsx')
has_translation_abs = (sum(relations_translations['rough translation'].notna()) / len(relations_translations)) * 100

# weighted by occurrence
counts = relations_translations['count']
has_translation_weighted = sum((relations_translations['rough translation'].notna() * 100) * counts) / sum(counts)

In [124]:
print(f'unweighted: {has_translation_abs:.2f}%')
print(f'weighted by frequency of occurrence: {has_translation_weighted:.2f}%')

unweighted: 74.26%
weighted by frequency of occurrence: 92.46%
